In [3]:
#packages
import numpy as np
import scipy as sp



In [5]:
class KMC_sim:
    def __init__(self, L):
        self.L = L # Lattice size
        self.c_B = 0.5 # Concentration of B atoms
        self.initialize_lattice()

    def initialize_lattice(self):
        num_B_atoms = int(self.L**3 *self.c_B)
        num_A_atoms = int(self.L**3 - num_B_atoms)
        lattice_values = np.array([-1] * num_B_atoms + [1] * num_A_atoms)
        np.random.shuffle(lattice_values)
        self.lattice = lattice_values.reshape((self.L, self.L, self.L))
        
        #Set the center atom to be a vacancy
        self.lattice[self.L//2, self.L//2, self.L//2] = 0
        self.vacancy_position = (self.L//2, self.L//2, self.L//2)

        #create lists and maps to store things
        self.atom_list = []
        self.atom_to_lattice_map = {}
        self.lattice_to_atom_map = {}
        self.atom_to_displacement_map = {}
        for index in range(self.L * self.L * self.L):
            i, j, k = index // self.L, index % self.L
            self.atom_list.append(self.lattice[i, j ,k])
            self.atom_to_lattice_map[index] = (i, j, k)
            self.lattice_to_atom_map[(i, j, k)] = index
            self.atom_to_displacement_map[index] = np.array[0,0,0]

    def calculate_msd(self):
        msd_A = np.mean([np.sum(disp ** 2) for index, disp in self.atom_to_displacement_map.items() if self.atom_list[index] == 1])
        msd_B = np.mean([np.sum(disp ** 2) for index, disp in self.atom_to_displacement_map.items() if self.atom_list[index] == -1])
        msd_vac = np.mean([np.sum(disp ** 2) for index, disp in self.atom_to_displacement_map.items() if self.atom_list[index] == 0])
        return msd_A, msd_B, msd_vac
    
    def initialize_parameters(self, T, E_aa, E_bb, E_ab, e0_a, e0_b, c_B):
        self.T = T
        self.E_aa = E_aa
        self.E_bb = E_bb
        self.E_ab = E_ab
        self.e0_a = e0_a
        self.e0_b = e0_b
        self.c_B = c_B
        #Total energy per atom
        self.total_energy_per_atom = self.get_total_energy()/(self.L**3)
        self.time = 0
    
    #Helper function to calculate energy contribution for an atom and its neighbors
     # Helper function to calculate energy contribution for a atom and its neighbors
    def calculate_energy_contribution(self, i, j, k):
        atom_type = self.lattice[i, j, k]
        # Neighbors in 4 directions with periodic boundary conditions
        neighbor_position_lists = [((i+1)%self.L, j, k), ((i-1)%self.L, j, k), (i, (j+1)%self.L, k), (i, (j-1)%self.L, k),
                                   (i, j, (k+1%self.L)), (i, j, (k-1%self.L))]
        E = 0
        for ni, nj, nk in neighbor_position_lists:
            neighbor_type = self.lattice[ni, nj]
            if atom_type == 0 or neighbor_type == 0:
                E += 0
            if atom_type == 1 and neighbor_type == 1:
                E += self.E_AA
            elif atom_type == -1 and neighbor_type == -1:
                E += self.E_BB
            else:
                E += self.E_AB
        return E
    
      #bond counting model
    def get_total_energy(self):
        E = 0
        for i in range(self.L):
            for j in range(self.L):
                for k in range(self.L):
                    E += self.calculate_energy_contribution(i,j,k)
        return E/2
    
     #calcuate the energy change if the vacancy and neighbor are swapped
    def get_energy_change(self, vacancy_pos, neighbor_pos):
        i_vac, j_vac, k_vac = vacancy_pos
        i_neigh, j_neigh, k_neigh = neighbor_pos

        dE_before = self.calculate_energy_contribution(i_neigh, j_neigh, k_neigh)
        self.lattice[i_vac, j_vac, k_vac], self.lattice[i_neigh, j_neigh, k_neigh] = self.lattice[i_neigh, j_neigh, k_neigh] , self.lattice[i_vac, j_vac, k_vac]
        dE_after = self.calculate_energy_contribution(i_vac, j_vac)
        self.lattice[i_vac, j_vac, k_vac], self.lattice[i_neigh, j_neigh, k_neigh] = self.lattice[i_neigh, j_neigh, k_neigh] , self.lattice[i_vac, j_vac, k_vac]
        return dE_after - dE_before
    
    def one_simulation_step(self):
        #pick two random atoms: pos_1 = (i1, j1, k1) and pos_2 = (i2, j2, k2)
        kB = sp.constants.value('Boltzman constant in eV/K')
        i_vac, j_vac, k_vac = self.vacancy_position
        neighbor_position_lists = [((i_vac + 1)% self.L, j_vac, k_vac), ((i_vac - 1)% self.L, j_vac, k_vac),
                                   (i_vac, (j_vac + 1)% self.L, k_vac), (i_vac, (j_vac - 1)%self.L, k_vac),
                                   (i_vac, j_vac, (k_vac + 1)% self.L), (i_vac, j_vac, (k_vac - 1)%self.L)]
        rates = []
        energy_changes = []
        for pos in neighbor_position_lists:
            dE = self.get_energy_change((i_vac, j_vac, k_vac), pos)
            migration_barrier = dE/2 + self.e0_A if self.lattice[pos] == 1 else dE/2+self.e0_B
            rates.append(np.exp(-migration_barrier/(kB*self.T)))
            energy_changes.append(dE)
        rates = np.array(rates)
        probabilities = rates / np.sum(rates)
        chosen_index = np.random.choice(len(neighbor_position_lists), p=probabilities)
        chosen_pos = neighbor_position_lists[chosen_index]
        dE = energy_changes[chosen_index]
        #swap the frequency with the chosen neighbor
        i_neigh, j_neigh, k_neigh = chosen_pos
        self._update_displacement(i_vac, j_vac, k_vac, i_neigh, j_neigh, k_neigh)
        self.lattice[i_vac, j_vac, k_vac], self.lattice[i_neigh, j_neigh, k_neigh] = self.lattice[i_neigh, j_neigh, k_neigh] , self.lattice[i_vac, j_vac, k_vac]
        self.vacancy_position = (i_neigh, j_neigh, k_neigh)
        self.total_energy_per_atom += dE / (self.L **3)
        self.time += np.random.uniform()/(np.sum(rates)*1e13)

    def _unwrap_positions(self, i1, j1, k1, i2, j2, k2):
        #unwrap the periodic boundary conditions
        dx = (i2 - i1 + self.L // 2) % self.L - self.L // 2
        dy = (j2 - j1 + self.L // 2) % self.L - self.L // 2
        dz = (k2 - k1 + self.L // 2) % self.L - self.L // 2
        return dx, dy, dz
    
    def _update_displacement(self, i_old, j_old, k_old, i_new, j_new, k_new):
        old_atom_index = self.lattice_to_atom_map[(i_old, j_old, k_old)]
        new_atom_index = self.lattice_to_atom_map[(i_new, j_new, k_new)]
        dx, dy, dz = self._unwrap_positions(i_old, j_old, k_old, i_new, j_new, k_new)
        self.atom_to_displacement_map[old_atom_index] += np.array([dx, dy, dz])
        self.atom_to_displacement_map[new_atom_index] -= np.array([dx, dy, dz])
        
        #Swap positions in the map
        self.atom_to_lattice_map[old_atom_index], self.atom_to_lattice_map[new_atom_index] = (i_new, j_new, k_new), (i_old, j_old, k_old)
        self.lattice_to_atom_map[(i_old, j_old, k_old)], self.lattice_to_atom_map[(i_new, j_new, k_new)] = new_atom_index, old_atom_index

    def calculate_warren_cowley_sro(self):
        # Calculate the Warren-Cowley short-range order parameter
        # The order parameter is defined as alpha_AB = 1 - (P_AB)/(2*c_A*c_B)
        n_total = self.L*self.L*2 # Total number of atoms
        n_AB = 0
        for i in range(self.L):
            for j in range(self.L):
                for k in range(self.L): # Only count the upper triangle to avoid double counting
                    atom_type = self.lattice[i, j]
                    if atom_type == 0:
                        continue
                # Neighbors in 6 directions with periodic boundary conditions
                neighbor_position_lists = [((i+1)%self.L, j), ((i-1)%self.L, j),
                                           (i, (j+1)%self.L), (i, (j-1)%self.L),
                                           (i, j, (k+1)%self.L)]
                for ni, nj in neighbor_position_lists:
                    neighbor_type = self.lattice[ni, nj]
                    if neighbor_type == 0:
                        continue
                    if atom_type != neighbor_type:
                        n_AB += 1
        p_AB = n_AB/n_total/2
        return 1 - 2*p_AB










